# 🚀 FreqAI Hybrid Strategy - GPU Backtest

**Simple Git-based workflow - No SSH, No tunnels!**

## 📋 Before you start:
1. ✅ Enable GPU: **Runtime > Change runtime type > T4 GPU**
2. ✅ Have your GitHub token ready (only if pushing logs)

## 🎯 What this does:
- Clones repo from GitHub
- Runs backtest on GPU
- Saves results to Google Drive

## ⏱️ Time:
- First run: ~5 min (install)
- Backtest: ~5-15 min

In [ ]:
# Check GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv

import torch
print(f"\n✅ GPU: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")

In [ ]:
# Clone repository
!rm -rf freqai-futures-strategy
print("📦 Cloning...")
!git clone https://github.com/aminak58/freqai-futures-strategy.git
%cd freqai-futures-strategy
print("\n✅ Done!")
!ls -lh

In [ ]:
# Install dependencies
print("📦 Installing... (2-3 min)\n")
!pip install -q ta-lib-binary
!pip install -q 'freqtrade[freqai]'
print("\n✅ Installed!")
!freqtrade --version

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

results_dir = '/content/drive/MyDrive/FreqAI_Results'
!mkdir -p "{results_dir}"
print(f"\n✅ Results → {results_dir}")

In [ ]:
# Optional: Load data from Drive
import os, zipfile

data_zip = '/content/drive/MyDrive/FreqAI/binance_data.zip'

if os.path.exists(data_zip):
    print("📦 Extracting data...")
    !mkdir -p user_data/data/binance
    with zipfile.ZipFile(data_zip) as z:
        z.extractall('user_data/data/')
    print("✅ Done!")
else:
    print("ℹ️  No data. Will download during backtest.")

In [ ]:
# 🔥 CRITICAL: Setup SOCKS5 Proxy to use local VPN
import os
import requests

print("🔌 Enter the SOCKS5 proxy URL from your local terminal:")
proxy_url = input()

if proxy_url.startswith("socks5h://"):
    os.environ['HTTP_PROXY'] = proxy_url
    os.environ['HTTPS_PROXY'] = proxy_url
    os.environ['http_proxy'] = proxy_url
    os.environ['https_proxy'] = proxy_url
    
    print("\n🌐 Testing proxy connection...")
    try:
        # We need to install pysocks for requests to use it
        !pip install -q pysocks
        
        response = requests.get("https://api.binance.com/api/v3/ping", timeout=20)
        if response.status_code == 200:
            print("✅✅✅ SUCCESS! Binance API is accessible!")
            print("🎯 Ready to backtest!\n")
        else:
            print(f"❌ FAILED! Status: {response.status_code}")
            print("   Check your local terminal and VPN.")
    except Exception as e:
        print(f"❌ FAILED! Error: {e}")
        print("   Is the local proxy script running?")
else:
    print("❌ Invalid URL. It must start with 'socks5h://'")

In [ ]:
# Run backtest (proxy should bypass Binance blocking)
print("🚀 Starting backtest...\n")
print("="*60)

STRATEGY = "FreqAIHybridStrategy"
TIMERANGE = "20250901-20251012"
PAIRS = "BTC/USDT:USDT"

print(f"Strategy: {STRATEGY}")
print(f"Timerange: {TIMERANGE}")
print(f"Pairs: {PAIRS}")
print("="*60)

import os
if 'HTTP_PROXY' in os.environ:
    print(f"\n✅ Using proxy: {os.environ['HTTP_PROXY']}")
else:
    print("\n⚠️  No proxy set - may fail!")

print("\n")

!freqtrade backtesting \
    --strategy {STRATEGY} \
    --config config/config.json \
    --freqaimodel LightGBMRegressorMultiTarget \
    --timerange {TIMERANGE} \
    --export trades \
    --datadir user_data/data/binance

print("\n✅ Backtest done!")

In [ ]:
# Save to Drive
from datetime import datetime
import os

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_folder = f"{results_dir}/backtest_{timestamp}"
!mkdir -p "{run_folder}"

print(f"💾 Saving...")

if os.path.exists('user_data/backtest_results'):
    !cp -r user_data/backtest_results/* "{run_folder}/"
    print(f"✅ Saved to: {run_folder}")
    !ls -lh "{run_folder}"
else:
    print("⚠️  No results found!")

---

## 🎉 Done!

### 📥 Access results:
- Go to Google Drive
- Navigate to `FreqAI_Results/`
- Download latest folder

### 🔁 Run again:
1. Edit parameters in Cell 7
2. Runtime > Run all

### 💡 Tips:
- First run: slower (installing)
- Later runs: faster (cached)
- Upload data to Drive to speed up

---